In [ ]:
import pandas as pd
import pyodbc
from fuzzywuzzy import fuzz
import datetime as dt

In [ ]:
username = 'vigrose'
password = 'Hufflepuff10946'

In [ ]:
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)

In [ ]:
sql_query = \
    '''
    SELECT DISTINCT 
    A.PARTY_ID, 
    A.POST_CD_ID,
    P.ADDR_1, 
    P.ADDR_2, 
    P.ADDR_3, 
    P.CITY, 
    S.SRC_STATE_CD AS STATE_CD, 
    P.POST_CD AS ZIP,
    A.FROM_DT,
    P.SRC_POST_KEY AS POST_KEY,
    C.PURPOSE_TYPE_DESC,
    C.PURPOSE_TYPE_ID,
    C.PURPOSE_USG_DESC
    FROM 
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE 
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    A.POST_CD_ID = P.POST_CD_ID 
    AND
    P.STATE_ID = S.STATE_ID
    AND 
    A.THRU_DT IS NULL
    ORDER BY A.FROM_DT DESC
    '''

In [ ]:
OTHER = pd.read_sql(con=AMAEDW, sql=sql_query)

In [ ]:
OTHER.head()

In [ ]:
customers = pd.read_excel('../../Data/Credentialling/Customers_pre-2020.xlsx')

In [ ]:
plz = pd.merge(customers, OTHER, on='PARTY_ID')

In [ ]:
plz = plz.fillna('None')

In [ ]:
plz = plz[plz['ORDER_PRODUCT_ID'].isin([4915513,4915521,4915502])]

In [ ]:
len(plz)

In [ ]:
plz.columns

In [ ]:
times =[]
for row in plz.itertuples():
    match = 0
    time_diff = 'None'
    if row.CUSTOMER_NAME == row.ADDR_3.upper():
        match = 1
    elif row.ZIP == row.LOCATION_ZIP or row.ZIP == row.MAIING_ZIP:
        match = 1
    elif fuzz.ratio(row.LOCATION_ADDRESS_1,row.ADDR_1.upper())>85:
        match = 1
    elif fuzz.ratio(row.MAILING_ADDRESS_1,row.ADDR_1.upper())>85:
        match = 1
    if match == 1:
        time_diff = row.FROM_DT - row.FULL_DT
        print(row.PURPOSE_USG_DESC)
        print(f'{row.ADDR_1.upper()} matches {row.MAILING_ADDRESS_1}/{row.LOCATION_ADDRESS_1}')
        print('---')
    times.append(time_diff)

In [ ]:
name_match

In [ ]:
plz['Time_Difference']=times

In [ ]:
matches = plz[plz['Time_Difference']!='None']

In [ ]:
len(matches)

In [ ]:
matches.groupby('PURPOSE_USG_DESC').count()[['CUSTOMER_NAME']]

In [ ]:
oldest = matches.sort_values('FROM_DT').drop_duplicates(['PARTY_ID','POST_CD_ID'], keep='first')

In [ ]:
dayz = list(oldest[oldest['Day_Difference']>0]['Day_Difference'])

In [ ]:
oldest['Day_Difference'] = [x.days for x in list(oldest['Time_Difference'])]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=dayz, bins=15, color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Days')
plt.ylabel('Frequency')
plt.title('Amount of Days until Address Appears in Record')
plt.text(23, 45, r'$\mu=15, b=3$')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)

In [ ]:
len(oldest[(oldest['Day_Difference']>0)&(oldest['Day_Difference']<=365)])

In [ ]:
len(oldest[oldest['Day_Difference']>0])

In [ ]:
oldest[(oldest['Day_Difference']>0)&(oldest['Day_Difference']<=365)].groupby('PURPOSE_TYPE_DESC').count().sort_values('CUSTOMER_NAME', ascending=False)

In [ ]:
plz.drop_duplicates([''])

In [ ]:
oldest[oldest['Day_Difference']>0]['Day_Difference'].median()

In [ ]:
customers['YEAR']=[x.year for x in list(customers['FULL_DT'])]

In [ ]:
customers[customers['ORDER_PRODUCT_ID'].isin([4915513,4915521,4915502])].groupby('YEAR').count()[['CUSTOMER_NAME']]

In [ ]:
oldest[oldest['Day_Difference']<0]

In [ ]:
oldest

In [ ]:
len(OTHER)

In [ ]:
plz.drop_duplicates(['PARTY_ID','CUSTOMER_NAME'])

In [ ]:
plz.columns

In [ ]:
matches.drop_duplicates(['CUSTOMER_NAME','PARTY_ID'])